In [ ]:
import cv2 as cv
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# Load the saved model
model_path = 'modelwithnoleaf.h5'
model = load_model(model_path)

# Function to preprocess and predict a single image
def preprocess_image(image):
    if image is None:
        return None
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image = cv.resize(image, (224, 224))  # Resize to the model's input size
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

def predict_and_display_label(image):
    processed_image = preprocess_image(image)
    if processed_image is None:
        return "Failed to preprocess", 0.0

    predictions = model.predict(processed_image)
    predicted_class = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    threshold = 0.5  # Set a threshold for confidence

    if confidence < threshold:
        label = "Not an image of leaves"
    else:
        # Updated to handle 4 categories
        if predicted_class == 0:
            label = "Healthy Arecanut Leaf"
        elif predicted_class == 1:
            label = "Yellow Leaf Disease"
        elif predicted_class == 2:
            label = "Other Leaf"
        else:
            label = "No Leaf Detected"
    
    return label, confidence

# Capture video from the camera (default camera index is 0)
cap = cv.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image from the camera. Exiting...")
        break

    # Get prediction and label for the current frame
    label, confidence = predict_and_display_label(frame)
    
    # Display the label and confidence on the frame
    display_text = f'{label} - Confidence: {confidence:.2f}'
    cv.putText(frame, display_text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv.LINE_AA)

    # Show the frame with prediction text
    cv.imshow('Camera Feed', frame)

    # Press 'q' to quit the loop and exit
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close any OpenCV windows
cap.release()
cv.destroyAllWindows()
